In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score,roc_auc_score, confusion_matrix, classification_report, accuracy_score

In [3]:
train = pd.read_csv("Healthcare-Diabetes.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2768 entries, 0 to 2767
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        2768 non-null   int64  
 1   Pregnancies               2768 non-null   int64  
 2   Glucose                   2768 non-null   int64  
 3   BloodPressure             2768 non-null   int64  
 4   SkinThickness             2768 non-null   int64  
 5   Insulin                   2768 non-null   int64  
 6   BMI                       2768 non-null   float64
 7   DiabetesPedigreeFunction  2768 non-null   float64
 8   Age                       2768 non-null   int64  
 9   Outcome                   2768 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 216.4 KB


In [5]:
columns_to_check = ['Outcome']  # Replace with your column names

for col in columns_to_check:
    if col in train.columns:
        print(f"Value percentages for column: {col}")
        value_counts = train[col].value_counts(normalize=True) * 100
        print(value_counts)
        print("\n")


Value percentages for column: Outcome
Outcome
0    65.606936
1    34.393064
Name: proportion, dtype: float64




In [7]:
X = train.drop(columns=['Outcome'])
y = train['Outcome']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=123)
# 50/50 of the temp into validation and test which corresponding to 15% and 15% of the original dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=123)

#X_res, y_res = rus.fit_resample(X_train, y_train)

# Convert data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test)

In [8]:
# where is fit() ???
# Train XGBoost model
xgb_model = xgb.train(
    {
        'max_depth': 5,
        'eta': 0.2,
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'hist',
    },
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dval, 'val')],
    early_stopping_rounds=500
)

# Predict on test set
y_pred = xgb_model.predict(dtest)
y_pred_binary = (y_pred >= 0.25).astype(int)


[0]	train-auc:0.89736	val-auc:0.87715
[1]	train-auc:0.92781	val-auc:0.91641
[2]	train-auc:0.93674	val-auc:0.91467
[3]	train-auc:0.94378	val-auc:0.91443
[4]	train-auc:0.94998	val-auc:0.91303
[5]	train-auc:0.95572	val-auc:0.91423
[6]	train-auc:0.96069	val-auc:0.91974
[7]	train-auc:0.96743	val-auc:0.92654
[8]	train-auc:0.97007	val-auc:0.92938
[9]	train-auc:0.97111	val-auc:0.93045
[10]	train-auc:0.97301	val-auc:0.93141
[11]	train-auc:0.97572	val-auc:0.93479
[12]	train-auc:0.97769	val-auc:0.93549
[13]	train-auc:0.97879	val-auc:0.93662
[14]	train-auc:0.98152	val-auc:0.94068
[15]	train-auc:0.98458	val-auc:0.94388
[16]	train-auc:0.98491	val-auc:0.94393
[17]	train-auc:0.98537	val-auc:0.94441
[18]	train-auc:0.98571	val-auc:0.94466
[19]	train-auc:0.98766	val-auc:0.94818
[20]	train-auc:0.98787	val-auc:0.94885
[21]	train-auc:0.98908	val-auc:0.95105
[22]	train-auc:0.98970	val-auc:0.95210
[23]	train-auc:0.99082	val-auc:0.95417
[24]	train-auc:0.99202	val-auc:0.95702
[25]	train-auc:0.99283	val-auc:0.95

In [9]:
# AUC Score
auc_score = roc_auc_score(y_test, y_pred)
print(f"AUC: {auc_score:.4f}")

# Calculate balanced accuracy with library 
balanced_accuracy_xgb = balanced_accuracy_score(y_test, y_pred_binary)
print(f"Balanced Accuracy (Sklearn): {balanced_accuracy_xgb:.4f}")

AUC: 0.9950
Balanced Accuracy (Sklearn): 0.9756
